In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tickerData import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from tensorflow import keras
from keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

KeyboardInterrupt: 

In [ ]:
df = getTickerData(ticker="btc-usd", period="max", interval="1d")
df.drop(columns={'Adj Close'}, inplace=True)
df.rename(columns={'Date':'date', 'Open':'open', 'High':'high','Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)
df = produce_prediction(df, 14, dropna=True)
df

[*********************100%***********************]  1 of 1 completed


,date,open,high,low,close,volume,pred
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,1.0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,1.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,1.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,1.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,1.0
...,...,...,...,...,...,...,...
3086,2023-02-28,23521.837891,23585.384766,23077.650391,23147.353516,20535363434,0.0
3087,2023-03-01,23150.929688,23880.632812,23088.626953,23646.550781,24662841200,0.0
3088,2023-03-02,23647.019531,23739.138672,23245.021484,23475.466797,20386398516,0.0
3089,2023-03-03,23476.632812,23479.347656,22213.238281,22362.679688,26062404610,0.0


In [ ]:
features = StandardScaler().fit_transform(df[df.columns.difference(['pred', 'date'])])
target = df['pred']

X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
x_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
x_train.shape

(2472, 5, 1)

In [ ]:
model = keras.Sequential() # initializing model

# input layer and LSTM layer with 50 neurons
model.add(LSTM(units=100, activation='relu',return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 100)            40800     
                                                                 
 dense (Dense)               (None, 5, 100)            10100     
                                                                 
 dropout (Dropout)           (None, 5, 100)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 81,151
Trainable params: 81,151
Non-traina

In [ ]:
history = model.fit(x_train, Y_train, epochs=5, batch_size=64,validation_split=0.2)

Epoch 1/5
31/31 [==============================] - 4s 29ms/step - loss: 0.6803 - accuracy: 0.5731 - val_loss: 0.7051 - val_accuracy: 0.5030
Epoch 2/5
31/31 [==============================] - 0s 10ms/step - loss: 0.6688 - accuracy: 0.6141 - val_loss: 1.1621 - val_accuracy: 0.4343
Epoch 3/5
31/31 [==============================] - 0s 10ms/step - loss: 0.6593 - accuracy: 0.6242 - val_loss: 2.0995 - val_accuracy: 0.4667
Epoch 4/5
31/31 [==============================] - 0s 10ms/step - loss: 0.6557 - accuracy: 0.6257 - val_loss: 1.9105 - val_accuracy: 0.5030
Epoch 5/5
31/31 [==============================] - 0s 10ms/step - loss: 0.6584 - accuracy: 0.6252 - val_loss: 1.8265 - val_accuracy: 0.5051


In [ ]:
x_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

# predicting target attribute on testing dataset
test_results = model.evaluate(x_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

20/20 [==============================] - 0s 2ms/step - loss: 2.2789 - accuracy: 0.5218
Test results - Loss: 2.278852701187134 - Accuracy: 52.18093991279602%
